# **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Cloning repo**

In [ ]:
%cd /content/
!rm -rf /content/yolov7
!git clone https://github.com/WongKinYiu/yolov7.git

# **Extract Data**

In [ ]:
%cd /content/yolov7
!rm -rf /content/yolov7/data
!mkdir /content/yolov7/data

In [ ]:
# Install unzip package
!apt install unzip -y

# Unzip dataset from Google Drive
# !unzip "/content/drive/MyDrive/colab2/roadsign_voc.zip" -d "/content/drive/MyDrive/colab/roadsign_voc/"
!unzip /content/drive/MyDrive/colab3/yolov7.zip -d /content/yolov7/data/
!mv /content/yolov7/data/yolov7/images/ /content/yolov7/data/
!mv /content/yolov7/data/yolov7/labels/ /content/yolov7/data/
!cp /content/drive/MyDrive/colab3/bosch.yaml /content/yolov7/data/
!rm -rf /content/yolov7/data/yolov7
# Copy data from Google Drive to Paddle Detection dataset folder

# **Installation**

In [ ]:
# %cd /content/drive/MyDrive/colab/yolov7
%cd /content/yolov7
!pip install -r requirements.txt
# !pip install -U torch==1.11.0
# !pip install -U torchvision==0.12.0
# !pip install -U numpy scipy tqdm addict imgaug==0.2.5 tensorboard==2.8 pycocotools thop

# **Download weight file**

In [ ]:
!mkdir /content/yolov7/weights
%cd /content/yolov7/weights
!wget https://github.com/meituan/YOLOv6/releases/download/0.1.0/yolov6t.pt

# **Quick Start on Drowsiness Dataset**

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvcc --version
!nvidia-smi
import torch
torch.zeros(1).cuda()

# **Backup folder link to Google Drive**

In [ ]:
# !rm -rf /content/drive/MyDrive/colab3/run/
# !mkdir /content/drive/MyDrive/colab3/run/
!rm -rf /content/yolov7/runs
!ln -s /content/drive/MyDrive/colab3/runs /content/yolov7/

# **Training code**

In [ ]:
%cd /content/YOLOv6
#training
# train p5 models
python train.py --workers 8 --device 0 --batch-size 32 --data data/bosch.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml

# train p6 models
python train_aux.py --workers 8 --device 0 --batch-size 16 --data data/bosch.yaml --img 1280 1280 --cfg cfg/training/yolov7-w6.yaml --weights '' --name yolov7-w6 --hyp data/hyp.scratch.p6.yaml

# resume training

# train p5 models
python -m torch.distributed.launch --nproc_per_node 4 --master_port 9527 train.py --workers 8 --device 0,1,2,3 --sync-bn --batch-size 128 --data data/bosch.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml

# train p6 models
python -m torch.distributed.launch --nproc_per_node 8 --master_port 9527 train_aux.py --workers 8 --device 0,1,2,3,4,5,6,7 --sync-bn --batch-size 128 --data data/bosch.yaml --img 1280 1280 --cfg cfg/training/yolov7-w6.yaml --weights '' --name yolov7-w6 --hyp data/hyp.scratch.p6.yaml

# **Evaluation**

In [ ]:
python test.py --data data/bosch.yaml --img 640 --batch 32 --conf 0.001 --iou 0.65 --device 0 --weights yolov7.pt --name yolov7_640_val

# **Inference**

In [ ]:
python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg

# **Export ONNX Model**

In [ ]:
!pip install onnx>=1.10.0
%cd /content/YOLOv6/weights
!wget https://github.com/meituan/YOLOv6/releases/download/0.1.0/yolov6t.onnx
%cd /content/YOLOv6/
!python ./deploy/ONNX/export_onnx.py \
    --weights yolov6s.pt \
    --img 640 \
    --batch 1
